In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
base_dir = r'C:\Projects\DeepfakeDetection\DeepFakeDetection_Dataset\data'

In [3]:
metadata_path = r"C:\Projects\DeepfakeDetection\DeepFakeDetection_Dataset\metadata.csv"
df = pd.read_csv(metadata_path)

In [4]:
df['video_path'] = df['video_path'].apply(lambda x: os.path.join(base_dir, x))

In [6]:
df

,video_path,label
0,C:\Projects\DeepfakeDetection\DeepFakeDetectio...,REAL
1,C:\Projects\DeepfakeDetection\DeepFakeDetectio...,FAKE
2,C:\Projects\DeepfakeDetection\DeepFakeDetectio...,REAL
3,C:\Projects\DeepfakeDetection\DeepFakeDetectio...,FAKE
4,C:\Projects\DeepfakeDetection\DeepFakeDetectio...,REAL
...,...,...
1995,C:\Projects\DeepfakeDetection\DeepFakeDetectio...,FAKE
1996,C:\Projects\DeepfakeDetection\DeepFakeDetectio...,REAL
1997,C:\Projects\DeepfakeDetection\DeepFakeDetectio...,FAKE
1998,C:\Projects\DeepfakeDetection\DeepFakeDetectio...,REAL


In [7]:
video_paths = df['video_path'].values
labels = df['label'].values

In [8]:
frame_height = 64
frame_width = 64
num_frames = 30

In [9]:
def extract_frames(video_path, num_frames, frame_height, frame_width):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    for i in range(num_frames):
        frame_id = int((i / num_frames) * total_frames)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (frame_width, frame_height))
        frames.append(frame)
    cap.release()
    return np.array(frames)

In [10]:
def process_videos(video_paths, num_frames, frame_height, frame_width):
    X, y = [], []
    for path, label in zip(video_paths, labels):
        frames = extract_frames(path, num_frames, frame_height, frame_width)
        if frames.shape[0] == num_frames:
            frames = frames / 255.0
            X.append(frames)
            y.append(label)
    return np.array(X), np.array(y)

X, y = process_videos(video_paths, num_frames, frame_height, frame_width)

In [11]:
X

array([[[[[0.47058824, 0.45490196, 0.52156863],
          [0.49803922, 0.47843137, 0.53333333],
          [0.51372549, 0.48235294, 0.55294118],
          ...,
          [0.41960784, 0.48235294, 0.67058824],
          [0.42352941, 0.48627451, 0.6745098 ],
          [0.42352941, 0.48627451, 0.6745098 ]],

         [[0.51372549, 0.49019608, 0.55686275],
          [0.54117647, 0.50980392, 0.58039216],
          [0.56078431, 0.52941176, 0.6       ],
          ...,
          [0.41960784, 0.48235294, 0.67058824],
          [0.42352941, 0.48627451, 0.6745098 ],
          [0.42352941, 0.48627451, 0.6745098 ]],

         [[0.55294118, 0.5254902 , 0.59607843],
          [0.57254902, 0.54117647, 0.61176471],
          [0.5372549 , 0.49803922, 0.58431373],
          ...,
          [0.42352941, 0.48235294, 0.67843137],
          [0.42352941, 0.48235294, 0.67843137],
          [0.42352941, 0.48235294, 0.67843137]],

         ...,

         [[0.94901961, 0.95294118, 0.9372549 ],
          [0.93333333,

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test) 
y_train = y_train.astype('float64')
y_test = y_test.astype('float64')

In [15]:
print(X_train.dtype)  
print(y_train.dtype)

float64
float64


In [21]:
history = model.fit(X_train, y_train, epochs=10, batch_size=4, validation_split=0.2)

Epoch 1/10
318/318 ━━━━━━━━━━━━━━━━━━━━ 207s 624ms/step - accuracy: 0.5283 - loss: 0.7241 - val_accuracy: 0.4702 - val_loss: 0.6943
Epoch 2/10
318/318 ━━━━━━━━━━━━━━━━━━━━ 198s 621ms/step - accuracy: 0.4654 - loss: 0.7141 - val_accuracy: 0.4702 - val_loss: 0.7024
Epoch 3/10
318/318 ━━━━━━━━━━━━━━━━━━━━ 196s 616ms/step - accuracy: 0.5086 - loss: 0.6974 - val_accuracy: 0.4702 - val_loss: 0.6960
Epoch 4/10
318/318 ━━━━━━━━━━━━━━━━━━━━ 192s 604ms/step - accuracy: 0.4826 - loss: 0.6997 - val_accuracy: 0.4702 - val_loss: 0.6936
Epoch 5/10
318/318 ━━━━━━━━━━━━━━━━━━━━ 191s 600ms/step - accuracy: 0.4893 - loss: 0.6962 - val_accuracy: 0.4702 - val_loss: 0.6944
Epoch 6/10
318/318 ━━━━━━━━━━━━━━━━━━━━ 192s 603ms/step - accuracy: 0.4971 - loss: 0.6973 - val_accuracy: 0.4702 - val_loss: 0.6947
Epoch 7/10
318/318 ━━━━━━━━━━━━━━━━━━━━ 193s 606ms/step - accuracy: 0.4761 - loss: 0.6962 - val_accuracy: 0.4702 - val_loss: 0.6952
Epoch 8/10
318/318 ━━━━━━━━━━━━━━━━━━━━ 191s 600ms/step - accuracy: 0.5153 -

In [17]:
y_train
X_train

array([[[[[0.18039216, 0.25490196, 0.3372549 ],
          [0.17647059, 0.25098039, 0.33333333],
          [0.19607843, 0.26666667, 0.36078431],
          ...,
          [0.4627451 , 0.51372549, 0.6745098 ],
          [0.45098039, 0.50588235, 0.65882353],
          [0.44705882, 0.50588235, 0.64705882]],

         [[0.16078431, 0.23529412, 0.31764706],
          [0.18823529, 0.2627451 , 0.34509804],
          [0.23137255, 0.30196078, 0.39607843],
          ...,
          [0.4627451 , 0.51372549, 0.67843137],
          [0.45098039, 0.50588235, 0.6627451 ],
          [0.44705882, 0.50196078, 0.65490196]],

         [[0.17254902, 0.24705882, 0.32941176],
          [0.21960784, 0.29411765, 0.37647059],
          [0.27058824, 0.34117647, 0.43921569],
          ...,
          [0.4627451 , 0.51372549, 0.67843137],
          [0.45098039, 0.50588235, 0.6627451 ],
          [0.44705882, 0.50196078, 0.65490196]],

         ...,

         [[0.40392157, 0.54509804, 0.72941176],
          [0.41176471,

In [18]:
def build_model(frame_height, frame_width, num_frames):
    model = Sequential()

    model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same'), input_shape=(num_frames, frame_height, frame_width, 3)))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu', padding='same')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Flatten()))

    model.add(LSTM(50, return_sequences=True))
    model.add(LSTM(50))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [19]:
model = build_model(frame_height, frame_width, num_frames)
model.summary()

c:\Users\abish\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed                │ (None, 30, 64, 64, 32) │           896 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 30, 32, 32, 32) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 30, 32, 32, 64) │        18,496 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 30, 16, 16, 64) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_4              │ (None, 30, 16384)      │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 30, 50)         │     3,287,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,326,643 (12.69 MB)

 Trainable params: 3,326,643 (12.69 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_acc}")

13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 220ms/step - accuracy: 0.4603 - loss: 0.6948
Test Loss: 0.6926705241203308
Test Accuracy: 0.5175879597663879


In [2]:
import numpy as np
import cv2
from tensorflow.keras.models import load_model

# Function to preprocess and extract frames from the video
def preprocess_video(video_path, num_frames, frame_height, frame_width):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    for i in range(num_frames):
        frame_id = int((i / num_frames) * total_frames)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (frame_width, frame_height))
        frames.append(frame)
    cap.release()
    
    # Convert frames to numpy array and normalize
    frames = np.array(frames)
    frames = frames / 255.0
    return np.expand_dims(frames, axis=0)  # Add a batch dimension

# Load the trained model
model = load_model('deepfake_detection_model.h5')

# Path to the new video 
video_path = 'C:\\Projects\\DeepfakeDetection\\DeepFakeDetection_Dataset\\data\\994_986.mp4'

# Preprocess the video
num_frames = 30  # The same number used during training
frame_height = 64
frame_width = 64
processed_video = preprocess_video(video_path, num_frames, frame_height, frame_width)

# Predict the class (Real or Fake)
prediction = model.predict(processed_video)
print("Prediction:", prediction)

# Interpret the result
if prediction >= 0.5:
    print("The video is predicted to be REAL.")
else:
    print("The video is predicted to be FAKE.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Prediction: [[0.50916773]]
The video is predicted to be REAL.


In [1]:
import numpy as np
import cv2
from tensorflow.keras.models import load_model

# Function to preprocess and extract frames from the video
def preprocess_video(video_path, num_frames, frame_height, frame_width):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    for i in range(num_frames):
        frame_id = int((i / num_frames) * total_frames)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (frame_width, frame_height))
        frames.append(frame)
    cap.release()
    
    # Convert frames to numpy array and normalize
    frames = np.array(frames)
    frames = frames / 255.0
    return np.expand_dims(frames, axis=0)  # Add a batch dimension

# Load the trained model
model = load_model('deepfake_detection_model.h5')

# Path to the new video



In [31]:
video_path = 'C:\\Projects\\DeepfakeDetection\\DeepFakeDetection_Dataset\\data\\001.mp4'

# Preprocess the video
num_frames = 30  # The same number used during training
frame_height = 64
frame_width = 64
processed_video = preprocess_video(video_path, num_frames, frame_height, frame_width)

# Predict the class (Real or Fake)
prediction = model.predict(processed_video)
print("Prediction:", prediction)

# Interpret the result
if prediction <= 0.5091603:
    print("The video is predicted to be Fake.")
else:
    print("The video is predicted to be Real.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
Prediction: [[0.50916034]]
The video is predicted to be Real.


In [1]:
model.save('deepfake_detection_model.h5')

NameError: name 'model' is not defined